In [ ]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff pyyaml facebook_business pystache

In [27]:
%load_ext autoreload         
%autoreload 2

In [1]:
from environs import Env

from adopt.malaria import get_confs, get_df, load_basics, window, get_db_conf
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState, get_api
from adopt.malaria import load_typed_json
from adopt.marketing import AudienceConf, CampaignConf
from adopt.campaign_queries import get_campaigns_for_user, create_campaign_confs, create_campaign_for_user
from typing import List
import json

env = Env()
env.read_env('.env-upswell-vlab', override=True)
db_conf = get_db_conf(env)

In [2]:
USER = "malaria2@vlab.digital"
CAMPAIGN = "mnm-jan-2021"
CAMPAIGNID = next(c['id'] for c in get_campaigns_for_user(USER, db_conf) 
                  if c['name'] == CAMPAIGN)

# create_campaign_for_user(USER, CAMPAIGN, db_conf)

In [3]:
userinfo, config, db_conf, state, m, confs = load_basics(CAMPAIGNID, env)

In [4]:
images = {i['name']: i for i in state.account.get_ad_images(fields=['name', 'hash'])}

# CREATING CONFS - MNM

In [5]:
# you have 10650 to spend

(75000000.0 / 7300)

10273.972602739726

In [6]:
c = {'optimization_goal': 'REPLIES',
     'destination_type': 'MESSENGER',
     'adset_hours': 48,
     'budget': 75000000.0,
     'min_budget': 10000.0,
     'opt_window': 7*24,
     'end_date': '2021-04-01',
     'page_id': '102998371752603',
     'instagram_id': None,
     'proportional': True,
     'ad_account': '2240415876083895',
     'ad_campaign': 'impact-evaluation-1shot-global-insights'}

config = CampaignConf(**c)

create_campaign_confs(CAMPAIGNID, "opt", [config._asdict()], db_conf)

('e49ba210-9928-4d9f-b62a-874c5652bb71',
 datetime.datetime(2021, 3, 26, 18, 11, 42, 897204, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
 'opt',
 None,
 [{'ad_account': '2240415876083895',
   'ad_campaign': 'impact-evaluation-1shot-global-insights',
   'adset_hours': 48,
   'budget': 75000000.0,
   'destination_type': 'MESSENGER',
   'end_date': '2021-04-01',
   'instagram_id': None,
   'min_budget': 10000.0,
   'opt_window': 168,
   'optimization_goal': 'REPLIES',
   'page_id': '102998371752603',
   'proportional': True}])

In [ ]:
import json
from adopt.marketing import make_audience_conf, dict_from_nested_type

audience_confs = [
  {
    "name": "vlab-mnm-kutcha",
    "shortcodes": ["extrabasehin1shot"],
    "subtype": "LOOKALIKE",
    "lookalike": {
      "name": "vlab-mnm-kutcha-lookalike",
      "target": 2000,
      "spec": {
        "country": "IN",
        "starting_ratio": 0.0,
        "ratio": 0.2
      }
    },
    "question_targeting": {
      "op": "equal",
      "vars": [
        {"type": "response", "value": "dwelling"},
        {"type": "constant", "value": "A"}
      ]
    }
  },
  {
    "name": "vlab-mnm-respondents-1shot-global-insights",
    "shortcodes": ["extrabasehin1shot"],
    "subtype": "CUSTOM"
  }
]



confs = [dict_from_nested_type(a) for a in audience_confs]

create_campaign_confs(CAMPAIGNID, "audience", confs, db_conf)

In [ ]:
from adopt.marketing import CreativeConf

def _creative_conf(name, image, body, form):
    return {
        "name": name,
        "image": image['name'],
        "image_hash": image['hash'],
        "body": body,
        "welcome_message": body,
        "link_text": "Messenger main survey shuroo karen!",
        "button_text": "Start!",
        "form": form,
    }

image_confs = [
    ('vlab-mnm-mother-daughter-voice-be-heard', '1.png', 'अपने विचार दुनिया को बताएं!'),
    ('vlab-mnm-family-flowers-keep-healthy', '2.png', 'हमें अपने स्वस्थ जीवन के राज बताएं और फ्री डाटा क्रेडिट जीतें।'),
    # ('vlab-mnm-schoolgirl-laptop-got-10-min', '3.png', 'क्या आपके पास सिर्फ १० मिनिट हैं? मोबाइल डाटा क्रेडिट में २०० रुपये पाएँ। आपकी राय हमारे लिए महत्वपूर्ण है!'),
    ('vlab-mnm-grandfather-bike-healthy', '4.png', 'अपने कम्यूनिटी को स्वस्थ रखने में हमें मदद करें- हमें बताएं आप क्या सोचते है!'),
    ('vlab-mnm-woman-sari-texting-got-time', '5.png', 'एक छोटे से सर्वे के लिए आपके पास टाइम है?'),
]

nncreatives = [CreativeConf(**_creative_conf(name, images[n], body, 'extrabasehin1shot')) for name, n, body in image_confs]
# creatives = load_typed_json("config/creatives.json", List[CreativeConf])
confs = [c._asdict() for c in creatives]

create_campaign_confs(CAMPAIGNID, "creative", confs, db_conf)

In [7]:
import pandas as pd
from facebook_business.adobjects.targetinggeolocationcustomlocation \
    import TargetingGeoLocationCustomLocation
from facebook_business.adobjects.targetinggeolocationcity import TargetingGeoLocationCity
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targeting import Targeting

from adopt.marketing import make_stratum_conf

def load_cities(path):
    cities = pd.read_csv(path)
    cities = cities[cities.rad >= 1.0]
    return cities

def create_location(lat, lng, rad):
    return {
        TargetingGeoLocationCustomLocation.Field.latitude: lat,
        TargetingGeoLocationCustomLocation.Field.longitude: lng,
        TargetingGeoLocationCustomLocation.Field.radius: rad,
        TargetingGeoLocationCustomLocation.Field.distance_unit: 'kilometer',
    }

def make_stratum(facebook_targeting, quota, df):
    locs = [create_location(r.lat, r.lng, r.rad) 
           for _, r in df.iterrows()]

    targeting = {
        **facebook_targeting,
        Targeting.Field.geo_locations: {
            TargetingGeoLocation.Field.location_types: ['home'],
            TargetingGeoLocation.Field.custom_locations: locs
        },
    }

    id_ = df.disthash.iloc[0]

    conf = { 'metadata': {'stratumid': id_},
             'facebook_targeting': targeting, 
             'creatives': ['vlab-mnm-mother-daughter-voice-be-heard',
                           'vlab-mnm-family-flowers-keep-healthy',
                           # 'vlab-mnm-schoolgirl-laptop-got-10-min',
                           'vlab-mnm-grandfather-bike-healthy',
                           'vlab-mnm-woman-sari-texting-got-time'],
             'shortcodes': ['extrabasehin', 'extrabasehin1shot']}
    

    base = {**conf, 
            'id': id_,
            'quota': int(quota * 0.7), 
            'audiences': [],
            'excluded_audiences': ["vlab-mnm-respondents-panel", "vlab-mnm-respondents-round-B", "vlab-mnm-kutcha-lookalike", "vlab-mnm-respondents-1shot", "vlab-mnm-respondents-1shot-global-insights"], 
            'question_targeting': {
                'op': 'and',
                'vars': [
                    { 'op': 'equal',
                      'vars': [{'type': 'response', 'value': 'md:stratumid'},
                               {'type': 'constant', 'value': id_}]},
                    { 'op': 'not_equal',
                      'vars': [{'type': 'response', 'value': 'dwelling'},
                               {'type': 'constant', 'value': 'A'}]},
                    {'op': 'or', 
                     'vars': [{ 'op': 'equal', 'vars':[{'type': 'translated_response', 'value': 'malaria4months'}, 
                                                       {'type': 'constant', 'value': 'Yes'}]}, 
                              {'op': 'equal', 'vars': [{'type': 'translated_response', 'value': 'fever4months'}, 
                                                       {'type': 'constant', 'value': 'Yes'}]}]}
                ]
            }}

    kutcha = {**conf, 
              'id': id_ + '-kutcha',
              'quota': int(quota * 0.3), 
              'audiences': ["vlab-mnm-kutcha-lookalike"],
              'excluded_audiences': ["vlab-mnm-respondents-panel", "vlab-mnm-respondents-round-B", "vlab-mnm-respondents-1shot", "vlab-mnm-respondents-1shot-global-insights"], 
              'question_targeting': {
                  'op': 'and',
                  'vars': [
                      { 'op': 'equal',
                        'vars': [{'type': 'response', 'value': 'md:stratumid'},
                                 {'type': 'constant', 'value': id_}]},
                      { 'op': 'equal',
                        'vars': [{'type': 'response', 'value': 'dwelling'},
                                 {'type': 'constant', 'value': 'A'}]}, 
                      {'op': 'or', 
                       'vars': [{ 'op': 'equal', 'vars':[{'type': 'translated_response', 'value': 'malaria4months'}, 
                                                         {'type': 'constant', 'value': 'Yes'}]}, 
                                {'op': 'equal', 'vars': [{'type': 'translated_response', 'value': 'fever4months'}, 
                                                         {'type': 'constant', 'value': 'Yes'}]}]}
                  ]
              }}

    return [base, kutcha]

In [ ]:
from adopt.marketing import dict_from_nested_type

df = load_cities('output/cities.csv')
df = df[['disthash', 'rad', 'lng', 'lat', 'creative_group', 'include_audience']]
strata = df.groupby('disthash').apply(lambda df: make_stratum({'age_min': 20}, 60, df)).tolist()

strata_data = [dict_from_nested_type(make_stratum_conf(c)) for s in strata for c in s]

create_campaign_confs(CAMPAIGNID, "stratum", strata_data, db_conf)

# TESTING

In [9]:
mal = load_basics(CAMPAIGNID, env)

In [ ]:
from adopt.malaria import update_audience_for_campaign, update_ads_for_campaign

instructions, report = update_ads_for_campaign(mal)

In [ ]:
report

In [ ]:
instructions, _ = update_audience_for_campaign(mal)

In [ ]:
from adopt.malaria import run_instructions

run_instructions(instructions, mal.state)

In [ ]:
from adopt.malaria import hydrate_strata, hydrate_audiences, get_df

userinfo, config, db_conf, state, m, confs = mal

# df = get_df(db_conf, userinfo.survey_user, audience_confs)

strata = hydrate_strata(state, confs["stratum"], confs["creative"])
df = get_df(db_conf, userinfo.survey_user, strata)

In [27]:
audience_confs = confs["audience"]
audiences = hydrate_audiences(df, m, audience_confs)

In [ ]:
strata = hydrate_strata(state, confs["stratum"], confs["creative"])
df = get_df(db_conf, userinfo.survey_user, strata)

In [16]:
import pandas as pd
from adopt.campaign_queries import get_last_adopt_report

rdf = pd.DataFrame(get_last_adopt_report(CAMPAIGNID, "FACEBOOK_ADOPT", mal.db_conf)).T

In [12]:
rdf = pd.DataFrame(report).T

In [13]:
from adopt.malaria import days_left

rdf['expected'] = (rdf.respondents + ((rdf.budget / rdf.price) * days_left(mal.config))).astype(int)
rdf['expected_share'] = rdf.expected / rdf.expected.sum()

In [ ]:
rdf.sort_values('budget', ascending=False)

In [ ]:
mal.state.total_spend / 73 + days_left(mal.config) * rdf.budget.sum() / 7300

In [23]:
rdf.expected.sum()

5659

In [22]:
rdf.respondents.sum()

5250.0

In [11]:
rdf['districtid'] = rdf.index.str.split('-').map(lambda x: x[0]).values

In [ ]:
rdf.groupby('districtid').agg({'expected': 'sum'}).reset_index().sort_values('expected')

In [ ]:
rdf.groupby('districtid').agg({'respondents': 'sum'}).reset_index().sort_values('respondents')

In [ ]:
# in this case, what I'd really like the opt to do
# is ensure a min representation of kutcha per district. 

# That sort of "soft" target probably only makes
# sense for lookalike variables, for hard demo variables
# you should always be able to get the exact amount you 
# want by turning on/off ads, you never get the wrong people

# But with lookalike targeting, you get the wrong people
# and you want some way to say "I'm a little flexible on
# the exact amount" -- i'm not sure if this makes sense
# actually in the big scheme - but probably. 

# Really, in this case, I probably want a different Kutcha
# target per district. 
#
# Or not. 

# The thing is this all adds a dependency between buckets
# the exact size of one bucket depends on another, the one
# that's the same in the hard demos but different in the implicits. 

# 

In [ ]:

with pd.option_context("display.max_rows", 160, "display.max_columns", 10):
    display(rdf.sort_values('budget', ascending=False))

In [35]:
import numpy as np

np.linalg.norm(rdf.expected_share - rdf.goal)

0.03000335994283512

In [36]:
np.linalg.norm(rdf.respondent_share - rdf.goal)

0.03685672725643937

In [99]:
t = (5000 / 80)

t*.7, t*.3

(43.75, 18.75)

In [97]:
rdf.expected.sum()

7252.580094737863

In [70]:
report.respondents.sum()

11669.0

In [ ]:
df.question_ref

In [78]:
rdf = pd.DataFrame(report).T

In [ ]:
rdf